# Purpose
This script will update the status of the provided contacts and their desired list to "Subscribed". Currently, there is no way to do this in bulk directly on ActiveCampaign.

### Requirements
* "contacts.csv" file with contact_id, list_name
* each contact should have been previously subscribed to the list - this is not an import
* chromedriver that matches your Chrome version (https://googlechromelabs.github.io/chrome-for-testing/)

In [19]:
import csv
import time
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options


In [12]:
activecampaign_subdomain = 'your_subdomain'
base_url = f'https://{activecampaign_subdomain}.activehosted.com/app/contacts/'

In [7]:
# Read contacts from csv file
def get_contact_ids():
    csv_file_path = 'contacts.csv'
    contacts = []

    # Read the CSV file
    with open(csv_file_path, mode='r') as file:
        csv_reader = csv.reader(file)
        
        # Skip the header row if there is one
        next(csv_reader, None)
        for row in csv_reader:
            contact_id = int(row[0])
            list_name = row[1]
            contacts.append({'contact_id': contact_id, 'list_name': list_name})

    return contacts

contacts = get_contact_ids()
# list of dictionaries
# {'contact_id': 2000, 'list_name': 'The list'}

In [8]:
def initialize_driver():
	# Set up the Chrome options
	chrome_options = Options()
	chrome_options.add_experimental_option("detach", True)  # To keep the browser open after the script finishes
	# chrome_options.headless = True  # Add this line to run Chrome in headless mode

	# Initialize the WebDriver
	# chrome_options.add_argument('--headless')
	# chrome_options.add_argument('--no-sandbox')
	driver = webdriver.Chrome(options=chrome_options)

	return driver

driver = initialize_driver()

In [13]:
# Navigate to the login page
driver.get(f'https://{activecampaign_subdomain}.activehosted.com/app/login')

# Log in manually by entering your username and password, and complete security prompt as required

In [22]:
# Iterate through every contact and set list status to Subscribed
for index, contact in enumerate(contacts[0:1]):
    print(index, contact)

    # open contact page
    driver.get(base_url + str(contact['contact_id']))
    
    # wait until page is loaded
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'posrel.contact-view-wrap.contacts_edit')))
    
    # open list status modal window based on contact's list name (multiple lists are possible for each contact)
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, f"//span[contains(@class, 'list-name contact-lists__list-item__name') and text() = '{contact['list_name']}']"))).click()
    time.sleep(1.0)

    # open status drop-down
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".ac_button.small.grey-border.popover-trigger.nobmargin"))).click()
    time.sleep(1.0)

    # click on "subscribed"
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".ac-icon-ok-circled"))).click()
    time.sleep(1.0)

    # close list status modal window
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "camp-button[variant='outline']"))).click()
    time.sleep(1.0)

0 {'contact_id': 1841, 'list_name': 'Private Label'}
